In [1]:
%pip install openpyxl

%pip install xlrd

import pandas as pd

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
file_name = "T2-24-25-Enrollments-after-Add-Drop-Process-v3.xls"
try:
    df = pd.read_excel(file_name, sheet_name="Course Roster Report", usecols=["Course","Section","OTHER/REGULAR-ELECTIVE/REGULAR-CORE","Course Variant","Student ID","Email ID","Batch","Primary Faculty"])
    print("Data successfully loaded into DataFrame.")
    df["Primary Faculty"] = df["Primary Faculty"].str.replace("Prof. ", "")
    df["Primary Faculty"] = df["Primary Faculty"].str.replace("Dr. ", "")
    df["Primary Faculty"] = df["Primary Faculty"].str.strip()

    df.to_csv("enrollment_master.csv", index=False)
    print("Filtered data successfully saved to 'enrollment_master.csv'.")
    
except Exception as e:
    print(f"An error occurred: {e}")


Data successfully loaded into DataFrame.
Filtered data successfully saved to 'enrollment_master.csv'.


In [3]:
file_name = "enrollment_master.csv"
df = pd.read_csv(file_name)
df1 =  pd.DataFrame()

df = df[df['OTHER/REGULAR-ELECTIVE/REGULAR-CORE'] != 'OTHER']
df1['Code'] = df['Course'].str.split('/').str[0]
df1['Title'] = df['Course'].str.split('/').str[1]
df1['Faculty'] = df['Primary Faculty']
df['CourseCode'] = df['Course'].str.split('/').str[0]
df['CourseTitle'] = df['Course'].str.split('/').str[1]

df1 = df1.drop_duplicates()
df = df.drop_duplicates()

df.drop('Course', axis=1, inplace=True)

df1.to_csv("course_master.csv", index=False)
df.to_csv("enrollment_master.csv", index=False)


In [4]:
df = pd.read_csv("enrollment_master.csv")
df['CourseCode'] = df['CourseCode'].str.strip()
df.to_csv("enrollment_master.csv", index=False)

df = pd.read_csv("course_master.csv")
df['Code'] = df['Code'].str.strip()
df.to_csv("course_master.csv", index=False)

In [5]:
file_name = "enrollment_master.csv"
df = pd.read_csv(file_name)
new_order = [
    'CourseCode',
    'CourseTitle',
    'OTHER/REGULAR-ELECTIVE/REGULAR-CORE',
    'Course Variant',
    'Student ID',
    'Batch',
    'Section',
    'Email ID',
    'Primary Faculty'
]

df = df[new_order]

df.rename(columns={'Email ID': 'EmailID'}, inplace=True)

df.to_csv(file_name, index=False)

In [6]:
# Add a new column called Lab in classroom_master.csv if 'LAB' string is found in the RoomNumber column

file_name = "classroom_master.csv"
df = pd.read_csv(file_name)

df['Lab'] = df['RoomNumber'].str.contains('LAB', case=False)
df.to_csv(file_name, index=False)



In [7]:
# Add a new column called DeptLab in classroom_master.csv if Lab column is True and 'R' string is found in the RoomNumber column

file_name = "classroom_master.csv"
df = pd.read_csv(file_name)

df['DeptLab'] = df['Lab'] & df['RoomNumber'].str.contains('R', case=False)
df.to_csv(file_name, index=False)

In [8]:
file_name = "enrollment_master.csv"
df = pd.read_csv(file_name)

for index,row in df.iterrows():
    if 'Sec' in row['Section']:
        df.at[index, 'Section'] = row['Section'].split('-')[-1]
    else:
        df.at[index, 'Section'] = 'A'

df.to_csv(file_name, index=False)

In [9]:
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri','Sat']
n_days = len(days)
times = ['08:00', '08:15', '08:30', '08:45',
         '09:00', '09:15', '09:30', '09:45',
         '10:00', '10:15', '10:30', '10:45',
         '11:00', '11:15', '11:30', '11:45',
         '12:00', '12:15', '12:30', '12:45', '13:00',
         '14:00', '14:15', '14:30', '14:45',
         '15:00', '15:15', '15:30', '15:45',
         '16:00', '16:15', '16:30', '16:45',
         '17:00', '17:15', '17:30', '17:45', '18:00']
n_times = len(times)

# classroom_cols = {'RoomNum': str, 'Capacity': int, 'Category': str,
#                   'HasProjector': bool, 'HasWacom': bool}
classroom_cols = {'RoomNumber': str, 'Capacity': int}
classrooms = pd.read_csv('classroom_master.csv', dtype=classroom_cols)
n_rooms = classrooms.shape[0]

#get room capacity given room id 
def get_room_capacity(room_id: int) -> int:
    return int(classrooms.loc[room_id, 'Capacity'])

def get_room_category(room_id: int) -> str:
    a=classrooms.loc[room_id, 'Lab']
    b=classrooms.loc[room_id, 'DeptLab']
    if a:
        if b:
            return 'CSE Lab'
        return 'ECE Lab'
    return 'NotLab'

def get_room_num(room_id: int) -> str:
    return classrooms.loc[room_id, 'RoomNumber']

def check_cse_lab_room(room_id: int) -> int:
    if get_room_category(room_id) == 'CSE Lab':
        return True
    return False

def check_ece_lab_room(room_id: int) -> int:
    if get_room_category(room_id) == 'ECE Lab':
        return True
    return False

def get_total_cse_lab_capacity() -> int:
    total_capacity = 0
    for i in range(n_rooms):
        if check_cse_lab_room(i):
            total_capacity += get_room_capacity(i)
    return total_capacity

def get_total_ece_lab_capacity() -> int:
    total_capacity = 0
    for i in range(n_rooms):
        if check_ece_lab_room(i):
            total_capacity += get_room_capacity(i)
    return total_capacity

def get_total_not_lab_capacity() -> int:
    total_capacity = 0
    for i in range(n_rooms):
        if not check_cse_lab_room(i) and not check_ece_lab_room(i):
            total_capacity += get_room_capacity(i)

def get_course_dept(course : str) -> str:
    course = course[0:3]
    if course =="ECE" or course =="VLS" or course =="COM":
        return "ECE"
    return "CSE"

courses = pd.read_csv('course_master.csv')
n_courses = courses.shape[0]
enrollment = pd.read_csv('enrollment_master.csv')
n_enrollments = enrollment.shape[0]
n_students = enrollment['EmailID'].nunique()
students = enrollment['EmailID'].unique()

course_id = {}
for i in range(n_courses):
    course_id[courses['Code'][i]] = i

course_id_code={}
for i in range(n_courses):
    course_id_code[i] = courses['Code'][i]

def get_course_id(course_code: str) -> int:
    return course_id[course_code.strip()]

def get_course_strength(course_id: int) -> int:
    return int(enrollment[enrollment['CourseCode'] == courses['Code'][course_id]].shape[0])

def get_courses_student(student_id: int) -> list:
    l =  enrollment[enrollment['EmailID'] == students[student_id]]['CourseCode'].values
    return [get_course_id(x) for x in l]
    
def get_course_title(course_id: int) -> str:
    return courses.loc[course_id, 'Title']
    

def get_course_category(course_id: int) -> str:
    
    a = courses.loc[course_id,'Lab']
    b = courses.loc[course_id,'Dept']
    if a:
        if b == 'CSE':
            return 'CSE Lab'
        return 'ECE Lab'
    return 'NotLab'

def get_dept(course_id: int) -> str:
    return courses.loc[course_id, 'Dept']

def get_not_rooms_course(course_id: int) -> list:
    l=[]
    for i in range(n_rooms):
        if get_room_category(i) != get_course_category(course_id):
            l.append(i)
    return l

def get_course_duration(course_id: int) -> int:
    return courses.loc[course_id, 'Exam Duration']


def get_preferred_slots(course_id: int) -> list:
    x = courses.loc[course_id, 'Preferred Timings'].split(',')
    triplets = []
    for i in range(0, len(x), 3):
        begin = x[i].strip(" [(\"'")
        end = x[i+1].strip(" )'\"]")
        day = x[i+2].strip(" )'\"]")
        triplets.append((begin, end, day))
    for i in range(len(triplets)):
        triplets[i] = (times.index(triplets[i][0]), times.index(triplets[i][1]), days.index(triplets[i][2]))
    return triplets


In [10]:
file_name = "courses_master.xlsx"
try:
    df = pd.read_excel(file_name, sheet_name="Exam Preferences", skiprows=[0])
    print("Data successfully loaded into DataFrame.")

    new_columns = [
        "Code", "Title", "Faculty1", "Faculty2", "Exam Type", "Duration (Hrs)",
        "2025-03-03 00:00:00", "2025-03-04 00:00:00", "2025-03-05 00:00:00",
        "2025-03-06 00:00:00", "2025-03-07 00:00:00", "2025-03-08 00:00:00",
        "Blank", 
        "09:30:00", "10:30:00", "11:30:00", "14:00:00", "15:00:00", "16:00:00", "17:00:00"
    ]

    num_new_cols = len(new_columns)
    if df.shape[1] < num_new_cols:
        raise Exception("The loaded DataFrame has fewer columns than expected.")
    df = df.iloc[:, :num_new_cols]

    df.columns = new_columns

    df.to_csv("exam_master.csv", index=False)
    print("Filtered data successfully saved to 'exam_master.csv'.")
    
except Exception as e:
    print(f"An error occurred: {e}")


hardcode_dates = [
    "2025-03-03 00:00:00", "2025-03-04 00:00:00", "2025-03-05 00:00:00",
    "2025-03-06 00:00:00", "2025-03-07 00:00:00", "2025-03-08 00:00:00"
]
hardcode_days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']
hardcode_times = ["09:30:00", "10:30:00", "11:30:00", "14:00:00", "15:00:00", "16:00:00", "17:00:00"]
file_path = "exam_master.csv"
df = pd.read_csv(file_path)

course_code_timings = {}
course_code_preftime = {}
course_code_prefdays = {}
course_code_type = {}

for _, row in df.iterrows():
    code = row["Code"]
    duration = float(row["Duration (Hrs)"]) * 60
    exam_type = str(row["Exam Type"])
    times_list = []
    days_list = []

    for t in hardcode_times:
        if row[t] == "Yes":
            times_list.append(t)
            
    for i in range(len(hardcode_dates)):
        if row[hardcode_dates[i]] == "Yes":
            days_list.append(hardcode_days[i])
    
    if code in course_code_timings:
        course_code_timings[code].append(duration)
    else:
        course_code_timings[code] = [duration]
    
    if code in course_code_type:
        course_code_type[code].append(exam_type)
    else:
        course_code_type[code] = [exam_type]
    
    if code in course_code_preftime:
        course_code_preftime[code].append(times_list)
    else:
        course_code_preftime[code] = [times_list]
    
    if code in course_code_prefdays:
        course_code_prefdays[code].append(days_list)
    else:
        course_code_prefdays[code] = [days_list]

# print(course_code_timings)
# print(course_code_prefdays)
# print(course_code_preftime)
# print(course_code_type)


# x = data.get_course_strength(data.get_course_id("DHS 101B"))
# print(x)

exam_choice = {}

for code, types in course_code_type.items():
    choice = "Same"
    if len(types) == 1:
        if types[0] == "** No Exam **":
            choice = "NoExam"
    else:
        if all(t == types[0] for t in types):
            if types[0] == "** No Exam **":
                choice = "NoExam"
            else:
                choice = "Same"
        else:
            choice = "Different"
    
    exam_choice[code] = choice
df_choice = pd.DataFrame(list(exam_choice.items()), columns=["code", "exm choice"])
df_choice.to_csv("exam_choice.csv", index=False)
print("exam_choice.csv written successfully.")

timing_choice = {}

for code, timings in course_code_timings.items():
    if len(timings) == 1 and exam_choice[code] == "Same":
        lab_dept = get_course_dept(code)
        if lab_dept == "CSE":
            lab_strength = get_total_cse_lab_capacity()
        else:
            lab_strength = get_total_ece_lab_capacity()
        if get_course_strength(get_course_id(code)) > lab_strength and course_code_type[code][0] == "Lab Exam":
            print("course:",code)
            print(course_code_type[code])
            choice = "Different"
        else:
            choice = "Same"
    else:
        choice = "Different"
    timing_choice[code] = choice
for code, t_choice in timing_choice.items():
    if code in exam_choice:
        if exam_choice[code] not in ["Different", "NoExam"]:
            exam_choice[code] = t_choice
    else:
        exam_choice[code] = t_choice

df_choice = pd.DataFrame(list(exam_choice.items()), columns=["code", "exam choice"])
df_choice.to_csv("exam_choice.csv", index=False)
print("exam_choice.csv written successfully.")


Data successfully loaded into DataFrame.
Filtered data successfully saved to 'exam_master.csv'.
exam_choice.csv written successfully.
course: DHS 101B
['Lab Exam']
exam_choice.csv written successfully.


/home/sarvesh/miniconda3/envs/timetabling/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [11]:
# Delete entries in 'enrollment_master.csv' file where the coursecode matches with code column in exam_choice.csv but the exam choice column is 'NoExam'ArithmeticError

df = pd.read_csv("enrollment_master.csv")

df1 = pd.read_csv("exam_choice.csv")

for index1, row1 in df1.iterrows():
    if df1.at[index1, 'exam choice'] == 'NoExam':
        for index, row in df.iterrows():
            if row['CourseCode'].strip() == row1['code'].strip():
                df.drop(index, inplace=True)

df.to_csv("enrollment_master.csv", index=False)




In [12]:
# Delete entries in 'enrollment_master.csv' file where the coursecode matches with code column in exam_choice.csv but the exam choice column is 'NoExam'ArithmeticError

df = pd.read_csv("exam_master.csv")

df1 = pd.read_csv("exam_choice.csv")

for index1, row1 in df1.iterrows():
    if df1.at[index1, 'exam choice'] == 'NoExam':
        for index, row in df.iterrows():
            if row['Code'].strip() == row1['code'].strip():
                df.drop(index, inplace=True)

df.to_csv("exam_master.csv", index=False)




In [13]:
import pandas as pd

df = pd.read_csv("enrollment_master.csv")
df2 = pd.read_csv("exam_master.csv")
exam_choice_df = pd.read_csv("exam_choice.csv")

choice_dict = {row['code'].strip(): row['exam choice'] for _, row in exam_choice_df.iterrows()}

course_dict = {}

for index, row in df.iterrows():
    code = row['CourseCode'].strip()
    section = row['Section'].strip()
    faculty = row['Primary Faculty']
    if code in choice_dict and choice_dict[code] != "Same":
        if '-' not in code:
            new_code = f"{code}-{section}"
            if code not in course_dict:
                course_dict[code] = {}
            if faculty in course_dict[code]:
                course_dict[code][faculty].append(new_code)
            else:
                course_dict[code][faculty] = [new_code]
            course_dict[code][faculty] = list(set(course_dict[code][faculty]))
            df.at[index, 'CourseCode'] = new_code

print(course_dict)

df.to_csv("enrollment_master.csv", index=False)
df2.to_csv("exam_master.csv", index=False)
print("Files updated successfully.")

{'CSE 102': {'Muralidhara V N': ['CSE 102-A'], 'Pradeesha Ashok': ['CSE 102-B']}, 'CSE 102 P': {'Muralidhara V N': ['CSE 102 P-A'], 'Vivek Yadav': ['CSE 102 P-B']}, 'CSE 104': {'Manisha Kulkarni': ['CSE 104-A'], 'Prem Singh': ['CSE 104-B']}, 'DAS 101': {'Uttam Kumar': ['DAS 101-B'], 'Raghuram Bharadwaj': ['DAS 101-A']}, 'DAS 101P': {'Vinu E V': ['DAS 101P-B'], 'Chandrashekar Ramanathan': ['DAS 101P-A']}, 'DHS 101B': {'Amit Prakash': ['DHS 101B-A', 'DHS 101B-B']}, 'DHS 201': {'Sumanth S': ['DHS 201-A'], 'TBA': ['DHS 201-B', 'DHS 201-A']}, 'EGC 121': {'Karthikeyan Vaidyanathan': ['EGC 121-A'], 'Ananda Y R': ['EGC 121-B']}, 'EGC 123': {'Ajay Bakre': ['EGC 123-A'], 'Amrita Mishra': ['EGC 123-B']}}
Files updated successfully.


In [14]:
save_dict = {}
for _, row in df2.iterrows():
    orig_code = row['Code'].strip()
    faculty1 = row['Faculty1'].strip() if isinstance(row['Faculty1'], str) else row['Faculty1']
    if orig_code not in save_dict:
        save_dict[orig_code] = {}
    save_dict[orig_code][faculty1] = row.to_dict()

for code, fac_dict in course_dict.items():
    for faculty in fac_dict.keys():
        df2 = df2[~((df2['Code'].str.strip() == code) & (df2['Faculty1'].str.strip() == faculty))]

new_rows = []
for code, fac_dict in course_dict.items():
    for faculty, new_codes in fac_dict.items():
        if code in save_dict and faculty in save_dict[code]:
            base_row = save_dict[code][faculty]
            for new_code in new_codes:
                new_row = base_row.copy()
                new_row['Code'] = new_code   
                new_row['Faculty1'] = faculty   
                new_rows.append(new_row)

if new_rows:
    new_rows_df = pd.DataFrame(new_rows)
    df2 = pd.concat([df2, new_rows_df], ignore_index=True)

df.to_csv("enrollment_master.csv", index=False)
df2.to_csv("exam_master.csv", index=False)
print("Files updated successfully.")


Files updated successfully.


In [15]:
import pandas as pd

hardcode_dates = [
    "2025-03-03 00:00:00", "2025-03-04 00:00:00", "2025-03-05 00:00:00",
    "2025-03-06 00:00:00", "2025-03-07 00:00:00", "2025-03-08 00:00:00"
]
hardcode_days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']
hardcode_times = ["09:30:00", "10:30:00", "11:30:00", "14:00:00", "15:00:00", "16:00:00", "17:00:00"]
file_path = "exam_master.csv"
df = pd.read_csv(file_path)

course_code_timings = {}
course_code_preftime = {}
course_code_prefdays = {}
course_code_type = {}

for _, row in df.iterrows():
    code = row["Code"]
    duration = float(row["Duration (Hrs)"]) * 60
    exam_type = str(row["Exam Type"])
    times_list = []
    days_list = []
    
    for t in hardcode_times:
        if row[t] == "Yes":
            times_list.append(t)
            
    for i in range(len(hardcode_dates)):
        if row[hardcode_dates[i]] == "Yes":
            days_list.append(hardcode_days[i])
    
    course_code_timings[code] = duration
    
    course_code_type[code] = exam_type
    
    course_code_preftime[code] = times_list
    
    course_code_prefdays[code] = days_list

print(course_code_timings)
print(course_code_prefdays)
print(course_code_preftime)
print(course_code_type)

{'AIM 101': 120.0, 'AIM 102': 180.0, 'AIM 704': 180.0, 'AIM 821': 180.0, 'AIM 831': 180.0, 'AIM 832': 180.0, 'AIM 843': 60.0, 'AIM 845': 180.0, 'AIM 846': 180.0, 'AMS 401': 180.0, 'AMS 402': 180.0, 'AMS 403': 150.0, 'AMS 404': 120.0, 'COM 605': 120.0, 'CSE 212': 120.0, 'CSE 606': 90.0, 'CSE 704': 150.0, 'CSE 718': 180.0, 'CSE 754': 180.0, 'CSE 816': 120.0, 'CSE 824': 180.0, 'CSE 836': 180.0, 'DAS 605': 120.0, 'DAS 839': 120.0, 'DHS 108': 180.0, 'DHS 110': 120.0, 'DHS 305': 120.0, 'DHS 306': 120.0, 'DHS 308': 180.0, 'DHS 314': 120.0, 'DHS 315': 120.0, 'DHS 611': 180.0, 'ECE 212': 180.0, 'ECE 212P': 180.0, 'ECE 303': 180.0, 'ECE 303P': 180.0, 'ECE 304': 90.0, 'EGC 301': 180.0, 'VLS 603': 180.0, 'VLS 804': 180.0, 'CSE 102-A': 180.0, 'CSE 102-B': 120.0, 'CSE 102 P-A': 150.0, 'CSE 102 P-B': 150.0, 'CSE 104-A': 120.0, 'CSE 104-B': 180.0, 'DAS 101-B': 180.0, 'DAS 101-A': 180.0, 'DAS 101P-B': 150.0, 'DAS 101P-A': 150.0, 'DHS 101B-A': 180.0, 'DHS 101B-B': 180.0, 'DHS 201-A': 180.0, 'DHS 201-B':

In [16]:
def get_duration(course_code):
    if course_code in course_code_timings:
        return int(course_code_timings[course_code])
    return None

In [17]:
import pandas as pd
import numpy as np
df = pd.read_csv("exam_master.csv")
n = len(df)

days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri','Sat']
times = ['08:00', '08:15', '08:30', '08:45',
         '09:00', '09:15', '09:30', '09:45',
         '10:00', '10:15', '10:30', '10:45',
         '11:00', '11:15', '11:30', '11:45',
         '12:00', '12:15', '12:30', '12:45', '13:00',
         '14:00', '14:15', '14:30', '14:45',
         '15:00', '15:15', '15:30', '15:45',
         '16:00', '16:15', '16:30', '16:45',
         '17:00', '17:15', '17:30', '17:45', '18:00']

lunch_time_index = times.index('13:00')
pre_lunch_times = list(range(lunch_time_index + 1))
post_lunch_times = list(range((lunch_time_index + 1), len(times)))

def begin_end_pairs(session_len: int) -> list:
    pairs = []
    for s in range(0, len(times) - session_len):
        e = s + session_len
        if (((s in pre_lunch_times) and (e in pre_lunch_times)) or
            ((s in post_lunch_times) and (e in post_lunch_times))):
            pairs.append((s, e))
    return pairs

def time_to_minutes(time_str):
    hh, mm = map(int, time_str.split(":"))
    return hh * 60 + mm

for idx, row in df.iterrows():
    duration = get_duration(row["Code"])
    session_len = duration // 15              
    pairs = begin_end_pairs(session_len)
    
    candidate_triplets = []
    for s, e in pairs:
        begin_time = times[s]
        end_time = times[e]
        if time_to_minutes(end_time) - time_to_minutes(begin_time) == duration:
            for day in days:
                candidate_triplets.append((begin_time, end_time, day))
    
    # print(f"Course {idx} with exam duration {duration} mins:")
    # print(candidate_triplets)
    # print("=" * 60)

In [18]:
def func(begin_time,pref_time):
    for x in pref_time:
        x = x[:-3]
        b1=time_to_minutes(begin_time)
        p1=time_to_minutes(x)
        if abs(b1-p1)<=15: # Allowing a difference of 15 minutes between the preferred time and the actual time
            return True
        
    return False

preferred_timings = []
for idx, row in df.iterrows():
    duration = get_duration(row["Code"]) 
    session_len = duration // 15                
    pairs = begin_end_pairs(session_len)
    pref_time = course_code_preftime[row["Code"]]
    pref_days = course_code_prefdays[row["Code"]]
    candidate_triplets = []
    for s, e in pairs:
        begin_time = times[s]
        end_time = times[e]
        if time_to_minutes(end_time) - time_to_minutes(begin_time) == duration:
            for day in days:
                if func(begin_time,pref_time) and day in pref_days:
                    candidate_triplets.append((begin_time, end_time, day))
    
    preferred_timings.append(candidate_triplets)

df["Preferred Timings"] = preferred_timings

df.to_csv("exam_master.csv", index=False)
    
    

In [19]:
df=pd.read_csv("exam_master.csv")
df=df[["Code","Title","Faculty1","Faculty2","Duration (Hrs)","Preferred Timings","Exam Type"]]

df.rename(columns={"Duration (Hrs)":"Exam Duration"},inplace=True)
df["Exam Duration"]=df["Exam Duration"]*60


df["Lab"]=df["Exam Type"].apply(lambda x: False if "Paper-Pen" in x or "On System" in x else True)

df["Dept"]=df["Code"].apply(lambda x: "ECE" if "ECE" in x or "VLS" in x or "COM" in x else "CSE")

df=df[["Code","Title","Faculty1","Faculty2","Exam Duration","Lab","Dept","Exam Type","Preferred Timings"]]

df.to_csv("course_master.csv",index=False)

In [20]:
courses = pd.read_csv('course_master.csv')
n_courses = courses.shape[0]
enrollment = pd.read_csv('enrollment_master.csv')
n_enrollments = enrollment.shape[0]
n_students = enrollment['EmailID'].nunique()
students = enrollment['EmailID'].unique()
print(n_students)

students_df = pd.DataFrame(students, columns=['EmailID'])
students_df.to_csv('students.csv', index=False)

973
